# Twist: ORBIT cloning_scheme

------


In [53]:
import pandas as pd
import Bio.Seq as Seq
import Bio.SeqIO

pd.options.display.max_colwidth = 200

The cloning scheme that we will use to get single stranded oligos with no PCR handle overhangs from the oligo pool comes from [this paper](https://pubs.acs.org/doi/10.1021/sb5001565) on 'MO-MAGE'. Note that there is detailed information in the supplement.

Their strategy was to amplify subpools of oligos as usual, but there are a few clever modifications. First, when amplifying, the **reverse primer is modified to include a 5' phosphate group**. This 5' end, i.e. the '-' strand will be selectively degraded by the lambda exonuclease ([neb link](https://www.neb.com/products/m0262-lambda-exonuclease). Also during that PCR it seems that the fwd primer has multiple 5' end PO bonds that selectively protect the '+' strand. 

To cleave off the PCR handles, they used a uracil in the fwd primer to introduce a site for the USER enzyme, and they included a DpnII site in the reverse primer sequence. By annealing just the reverse primer you can create a double stranded template for DpnII, which will cleanly cleave off the 5' end of the '+' strand. See the diagram below:

<img align="center" src="mo_mage_diagram.png" alt="drawing" width="500"/>

In this way, they got ssDNA oligos with no overhangs. 

I will modify their approach slightly, mainly because the USER enzyme is quite expensive, and I would like to avoid it if possible. For my cloning scheme I will introduce two restriction sites: [bsrI](https://www.neb.com/products/r0527-bsri) and [dpnII](https://www.neb.com/products/r0543-dpnii). DpnII seemed to work well enough in their scheme for cleaving the 3'end. I chose BsrI for the 5' end because I can still include a T as the 3' overhang, allowing us to use the USER enzyme if this approach fails. 

<img src="dpnII.png" alt="drawing" width="500"/><img src="bsrI.png" alt="drawing" width="500"/>

Further, both of these enzymes are quite cheap, and they both work in the NEB 3.1 buffer, although they require different temperature (37 vs. 65) that will require a two step protocol...but at least we don't need to switch buffers. Note that there is potential STAR activity for DpnII, but there should only be 1 possible cut site, so I think that's fine. I think that many different restriction enzymes could work for these two sites, for example [nlaIII](https://www.neb.com/products/r0125-nlaiii) or [bsmI](https://www.neb.com/products/r0134-bsmi) could work for the 5' site as well, but it wouldn't allow us to cleanly include a 'T' as a backup plan...but good to keep in mind for the future. 

For short recognition sequences, we can actually just find orthogonal kosuri primers that have the desired restriction site sequences. It would actually benefit this particular order to have longer sequences to be closer in length to the reg-seq constructs. The only issue is that when we purify / clean up this reaction we will be trying to purify a 128 bp oligo from 20 bp oligo, which already may be difficult. The longer those flanking oligos are the harder that step may be...that said these pcr handles should have no homology to genome and hopefully wouldn't affect anything even if they are electroporated directly into cell. 

So, this notebook will find orthogonal primers that match our restriction sites (and add a T to bsrI site) to append to the ORBIT sequences of interest.

------

# Generate Orthogonal primers with RE sites

First, let's read in the fwd and reverse orthogonal primers. We'll go ahead and reverse complement these sequences since we will need them in that format to append to the ORBIT sequences.

In [54]:
df_rev = pd.DataFrame()
i = 0

for record in Bio.SeqIO.parse("reverse_finalprimers.fasta", "fasta"):
    df_rev.loc[i,'rev_seq']=str(record.seq)
    df_rev.loc[i, 'rev_seq_comp']=str(record.seq.reverse_complement())
    i = i+1

In [55]:
df_rev

,rev_seq,rev_seq_comp
0,AAGTATCTTTCCTGTGCCCA,TGGGCACAGGAAAGATACTT
1,TGGTAGTAATAAGGGCGACC,GGTCGCCCTTATTACTACCA
2,AGGGGTATCGGATACTCAGA,TCTGAGTATCCGATACCCCT
3,ATCGATTCCCCGGATATAGC,GCTATATCCGGGGAATCGAT
4,TACTAACTGCTTCAGGCCAA,TTGGCCTGAAGCAGTTAGTA
...,...,...
2995,GTCCGTGTAGGATCGCCTTT,AAAGGCGATCCTACACGGAC
2996,GACTCTAGTGCGGGTGGTAC,GTACCACCCGCACTAGAGTC
2997,TTGACCAGGGTAAGCCGATC,GATCGGCTTACCCTGGTCAA
2998,GATTCAAGACGGCACTCGGA,TCCGAGTGCCGTCTTGAATC


Looks good. Now let's look for specific primers that end with the DpnII recognition site *GATC*.

In [56]:
df_rev_DpnII = df_rev.loc[df_rev['rev_seq'].str.endswith('GATC', na = False)]
df_rev_DpnII

,rev_seq,rev_seq_comp
349,CCAACCAGAATCGAACGATC,GATCGTTCGATTCTGGTTGG
468,GTGACATCACACGGTTGATC,GATCAACCGTGTGATGTCAC
527,AAGAGGGTCGTATTCCGATC,GATCGGAATACGACCCTCTT
861,CAGCTTTTGGACGATGGATC,GATCCATCGTCCAAAAGCTG
1584,AAAGCCCCACGGAATTGATC,GATCAATTCCGTGGGGCTTT
1695,TCCGGCTCTCCCTTAAGATC,GATCTTAAGGGAGAGCCGGA
1856,CGGCTAAGTGAAGTCCGATC,GATCGGACTTCACTTAGCCG
1888,AACGGCAGGGATGAAAGATC,GATCTTTCATCCCTGCCGTT
1910,ATCTTCGGAGGGGAGAGATC,GATCTCTCCCCTCCGAAGAT
2389,GGCCGTTTAAGGGATCGATC,GATCGATCCCTTAAACGGCC


Ok, there are about 10 different reverse primers that contain the restriction site. 

Now let's read in the fwd primers.

In [57]:
df_fwd = pd.DataFrame()
i = 0

for record in Bio.SeqIO.parse("forward_finalprimers.fasta", "fasta"):
    df_fwd.loc[i,'fwd_seq']=str(record.seq)
    #df_fwd.loc[i, 'fwd_rev_comp']=str(record.seq.reverse_complement())
    i = i+1

In [58]:
df_fwd

,fwd_seq
0,ATATAGATGCCGTCCTAGCG
1,CCCTTTAATCAGATGCGTCG
2,TTGGTCATGTGCTTTTCGTT
3,GGGTGGGTAAATGGTAATGC
4,TCCGACGGGGAGTATATACT
...,...
2995,GTCGATCACCGCCCCTTTTA
2996,CACGGAGGCAGCAAGACTTA
2997,AGGTCGAAGTGTCGCGTAAA
2998,TGTGCACTATCGATCACGGG


Let's look for the BsrI recognition site *ACTGG*.

In [59]:
df_fwd_BsrI = df_fwd.loc[df_fwd['fwd_seq'].str.endswith('ACTGG', na = False)]

df_fwd_BsrI

,fwd_seq
390,TTAATCTTAGGCCCCACTGG
742,AGATTAGCTGCCGATACTGG
790,GATCCTTGACTACCGACTGG
1026,ATTTCTCCACTCCCAACTGG
1490,AATGTCTTGCCCCTTACTGG
2090,AAATCTTTGCCCTCCACTGG
2473,TAAGCCCAATCTCCCACTGG


Ok, there are 7 fwd primers that contain this ending sequence. 

Now let's add a 'T' to the 3' end so that we can have a T in case we need to use the USER enzyme. This nt will form the final nt in the BsrI recognition site, and cleavage occurs directly after this T, hopefully leaving only our ORBIT sequence of interest.

In [60]:
df_fwd_BsrI['fwd_seq_t'] = df_fwd['fwd_seq'] + 'T'

df_fwd_BsrI

<ipython-input-60-5abdfba3f52e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fwd_BsrI['fwd_seq_t'] = df_fwd['fwd_seq'] + 'T'


,fwd_seq,fwd_seq_t
390,TTAATCTTAGGCCCCACTGG,TTAATCTTAGGCCCCACTGGT
742,AGATTAGCTGCCGATACTGG,AGATTAGCTGCCGATACTGGT
790,GATCCTTGACTACCGACTGG,GATCCTTGACTACCGACTGGT
1026,ATTTCTCCACTCCCAACTGG,ATTTCTCCACTCCCAACTGGT
1490,AATGTCTTGCCCCTTACTGG,AATGTCTTGCCCCTTACTGGT
2090,AAATCTTTGCCCTCCACTGG,AAATCTTTGCCCTCCACTGGT
2473,TAAGCCCAATCTCCCACTGG,TAAGCCCAATCTCCCACTGGT


With that we can concatenate the fwd and reverse primer dataframes.

In [61]:
df_fwd_rev = pd.concat([df_fwd_BsrI.reset_index(drop = True), df_rev_DpnII[0:7].reset_index(drop = True)], axis = 1, sort = False)
df_fwd_rev

,fwd_seq,fwd_seq_t,rev_seq,rev_seq_comp
0,TTAATCTTAGGCCCCACTGG,TTAATCTTAGGCCCCACTGGT,CCAACCAGAATCGAACGATC,GATCGTTCGATTCTGGTTGG
1,AGATTAGCTGCCGATACTGG,AGATTAGCTGCCGATACTGGT,GTGACATCACACGGTTGATC,GATCAACCGTGTGATGTCAC
2,GATCCTTGACTACCGACTGG,GATCCTTGACTACCGACTGGT,AAGAGGGTCGTATTCCGATC,GATCGGAATACGACCCTCTT
3,ATTTCTCCACTCCCAACTGG,ATTTCTCCACTCCCAACTGGT,CAGCTTTTGGACGATGGATC,GATCCATCGTCCAAAAGCTG
4,AATGTCTTGCCCCTTACTGG,AATGTCTTGCCCCTTACTGGT,AAAGCCCCACGGAATTGATC,GATCAATTCCGTGGGGCTTT
5,AAATCTTTGCCCTCCACTGG,AAATCTTTGCCCTCCACTGGT,TCCGGCTCTCCCTTAAGATC,GATCTTAAGGGAGAGCCGGA
6,TAAGCCCAATCTCCCACTGG,TAAGCCCAATCTCCCACTGGT,CGGCTAAGTGAAGTCCGATC,GATCGGACTTCACTTAGCCG


And finally let's clean it up just to the sequences we will append to the beginning (`fwd_seq_t`) and end (`rev_seq_comp`) of the ORBIT oligos. In the future, we can return to this notebook to get the actual primer sequences we will use to amplify the ORBIT constructs.

In [62]:
df_fwd_rev = df_fwd_rev[['fwd_seq_t', 'rev_seq_comp']]

df_fwd_rev

,fwd_seq_t,rev_seq_comp
0,TTAATCTTAGGCCCCACTGGT,GATCGTTCGATTCTGGTTGG
1,AGATTAGCTGCCGATACTGGT,GATCAACCGTGTGATGTCAC
2,GATCCTTGACTACCGACTGGT,GATCGGAATACGACCCTCTT
3,ATTTCTCCACTCCCAACTGGT,GATCCATCGTCCAAAAGCTG
4,AATGTCTTGCCCCTTACTGGT,GATCAATTCCGTGGGGCTTT
5,AAATCTTTGCCCTCCACTGGT,GATCTTAAGGGAGAGCCGGA
6,TAAGCCCAATCTCCCACTGGT,GATCGGACTTCACTTAGCCG


# Add primer sequences to ORBIT oligos

Now let's actually make our final TWIST constructs that contain our PCR handles, RE sites, and ORBIT targeting oligo.

In [63]:
df_1 = pd.read_csv("twist_orbit_tf_del_FL_short.csv")
df_2 = pd.read_csv("twist_orbit_tf_del_FL_long.csv")

df_3 = pd.read_csv("twist_orbit_tf_del_AO_short.csv")
df_4 = pd.read_csv("twist_orbit_tf_del_AO_long.csv")

In [64]:
ends_1 = df_fwd_rev.iloc[0,:].str.lower()

df_1['sequence'] = ends_1['fwd_seq_t'] + df_1['oligo'] + ends_1['rev_seq_comp']

df_1['pool_name'] = 'orbit_TF_del_first_last_short'
df_1['primers_added'] = True

df_1_clean = df_1[['sequence','pool_name','primers_added']]
df_1_clean

,sequence,pool_name,primers_added
0,ttaatcttaggccccactggtAATCTCTCTGCAACCAAAGTGAACCAATGAGAGGCAACAAGAATGggcttgtcgacgacggcggtctccgtcgtcaggatcatTGAGGGTGTTACATGAATTCATACTCAATTGCTGTCATCGGAGTGgatcgttcgattctggttgg,orbit_TF_del_first_last_short,True
1,ttaatcttaggccccactggtTATGCACAATAATGTTGTATCAACCACCATATCGGGTGACTTATGggcttgtcgacgacggcggtctccgtcgtcaggatcatTAATCTCTGCCCCGTCGTTTCTGACGGCGGGGAAAATGTTGCTTAgatcgttcgattctggttgg,orbit_TF_del_first_last_short,True
2,ttaatcttaggccccactggtGATGAATGAGTTTTCTATAAACTTATACTTAATAATTAGAAGTTAatgatcctgacgacggagaccgccgtcgtcgacaagccCATGGTAACCTCTCATCTTACTTATGAAATTTTAATGTATTCTGTgatcgttcgattctggttgg,orbit_TF_del_first_last_short,True
3,ttaatcttaggccccactggtGCTTCGAAGAGAGACACTACCTGCAACAATCAGGAGCGCAATATGggcttgtcgacgacggcggtctccgtcgtcaggatcatTAAAAATTTAGCTAAACACATATGAATTTTCAGATGTGTTTTATCgatcgttcgattctggttgg,orbit_TF_del_first_last_short,True
4,ttaatcttaggccccactggtGGCTAAAATAGAATGAATCATCAATCCGCATAAGAAAATCCTATGggcttgtcgacgacggcggtctccgtcgtcaggatcatTGATCGGCTTTTTTAATCCCATACTTTTCCACAGGTAGATCCCAAgatcgttcgattctggttgg,orbit_TF_del_first_last_short,True
...,...,...,...
69,ttaatcttaggccccactggtTAAGGGCATCTGTTTTTTATATTCAAGAATGAAAAATTTTTGTCAatgatcctgacgacggagaccgccgtcgtcgacaagccCATTACCAATACCTTACATATATTACTCATTAATGTATGTGCGAAgatcgttcgattctggttgg,orbit_TF_del_first_last_short,True
70,ttaatcttaggccccactggtATATGAGTGTCGAATCCTTATCCAAAACAAGAGGTAACTCTCATGggcttgtcgacgacggcggtctccgtcgtcaggatcatTGAACAAATTTTATCAGGTGACGTTCCGTAAAAAGTTGTATGGAGgatcgttcgattctggttgg,orbit_TF_del_first_last_short,True
71,ttaatcttaggccccactggtAGCCATGCACCGTAGACCAGATAAGCTCAGCGCATCCGGCAGTTAatgatcctgacgacggagaccgccgtcgtcgacaagccCATTTCATACTTACCTTTTTGTACGTACTTACTAAAAGTAAGTTTgatcgttcgattctggttgg,orbit_TF_del_first_last_short,True
72,ttaatcttaggccccactggtGGTTATTTAACGGCGCGAGTGTAATCCTGCCAGTGCAAAAAATCAatgatcctgacgacggagaccgccgtcgtcgacaagccCATACATACTCCACTAGTTATCGTTGATTTTGTCCAACAACTTGTgatcgttcgattctggttgg,orbit_TF_del_first_last_short,True


In [65]:
ends_2 = df_fwd_rev.iloc[1,:].str.lower()

df_2['sequence'] = ends_2['fwd_seq_t'] + df_2['oligo'] + ends_2['rev_seq_comp']

df_2['pool_name'] = 'orbit_TF_del_first_last_long'
df_2['primers_added'] = True

df_2_clean = df_2[['sequence','pool_name','primers_added']]
df_2_clean

,sequence,pool_name,primers_added
0,agattagctgccgatactggtCTATATTATGTGATCTAAATCACTTTTAAGTCAGAGTGAATAATGggcttgtcgacgacggcggtctccgtcgtcaggatcatTAATTCATATTGTACTGTTACGTTGTACAAACCTGTGCCAACGGGgatcaaccgtgtgatgtcac,orbit_TF_del_first_last_long,True
1,agattagctgccgatactggtGAGTCTGGCGGATGTCGACAGACTCTATTTTTTTATGCAGTTTTAatgatcctgacgacggagaccgccgtcgtcgacaagccCATGACGCCACCGATAACCGTTATTTATCAGACCAAAGAAACTGGgatcaaccgtgtgatgtcac,orbit_TF_del_first_last_long,True
2,agattagctgccgatactggtCGACGAAAATGTCCAGGAAAAATCCTGGAGTCAGATTCAGGGTTAatgatcctgacgacggagaccgccgtcgtcgacaagccCATATGTTCGTGAATTTACAGGCGTTAGATTTACATACATTTGTGgatcaaccgtgtgatgtcac,orbit_TF_del_first_last_long,True
3,agattagctgccgatactggtGTGGCTCTTGCCACGGTTCAGCATCGGCAAACAGATCCAACATTAatgatcctgacgacggagaccgccgtcgtcgacaagccCATAATCAGCTCCCTGGTTAAGGATAGCCTTTAGGCTGCCCGGTCgatcaaccgtgtgatgtcac,orbit_TF_del_first_last_long,True
4,agattagctgccgatactggtTTAGCGAGAACTGGTCTTTTATTCGCACTCAGGAGTACATGTATGggcttgtcgacgacggcggtctccgtcgtcaggatcatTGATTTTTAACCTTAACGAAGAGCTATATTAATAACGGCATCAGCgatcaaccgtgtgatgtcac,orbit_TF_del_first_last_long,True
...,...,...,...
221,agattagctgccgatactggtAAAGAATTTCGCCAGTTAATGCATCTTTAATCGGGAACTTTCATGggcttgtcgacgacggcggtctccgtcgtcaggatcatTAACGTCAGAAGGTTAATTCTGTTTCCAGCAGCGTCAGGATACTTgatcaaccgtgtgatgtcac,orbit_TF_del_first_last_long,True
222,agattagctgccgatactggtCGCGGAATAATCACGCAATTAACTAAACAAGGTTTAGTGAAGATGggcttgtcgacgacggcggtctccgtcgtcaggatcatTGATGGCGCGATAACGTAGAAAGGCTTCCCGAAGGAAGCCTTGATgatcaaccgtgtgatgtcac,orbit_TF_del_first_last_long,True
223,agattagctgccgatactggtCTATGTGATCTCCATTTCGATTGATTTAGTGTTTATTGACGTATGggcttgtcgacgacggcggtctccgtcgtcaggatcatTGATTATAAAAAAAACTTATTATTTATTTTAGTTTTTATCAGTGGgatcaaccgtgtgatgtcac,orbit_TF_del_first_last_long,True
224,agattagctgccgatactggtTGACGATTTTCCCCGTTCCCGGTTGCTGTACCGGGAACGTATTTAatgatcctgacgacggagaccgccgtcgtcgacaagccCATTTCTCCAGCACTCTGGAGAAATAGGCAAGACATTGGCAGAAAgatcaaccgtgtgatgtcac,orbit_TF_del_first_last_long,True


In [66]:
ends_3 = df_fwd_rev.iloc[2,:].str.lower()

df_3['sequence'] = ends_3['fwd_seq_t'] + df_3['oligo'] + ends_3['rev_seq_comp']

df_3['pool_name'] = 'orbit_TF_del_avd_ovlp_short'
df_3['primers_added'] = True

df_3_clean = df_3[['sequence','pool_name','primers_added']]
df_3_clean

,sequence,pool_name,primers_added
0,gatccttgactaccgactggtCTCTCTGCAACCAAAGTGAACCAATGAGAGGCAACAAGAATGAACggcttgtcgacgacggcggtctccgtcgtcaggatcatCAACGCTGTAAACTTATTTGAGGGTGTTACATGAATTCATACTCAgatcggaatacgaccctctt,orbit_TF_del_avd_ovlp_short,True
1,gatccttgactaccgactggtGCACAATAATGTTGTATCAACCACCATATCGGGTGACTTATGCGAggcttgtcgacgacggcggtctccgtcgtcaggatcatCTGTTCGACCAGGAGCTTTAATCTCTGCCCCGTCGTTTCTGACGGgatcggaatacgaccctctt,orbit_TF_del_avd_ovlp_short,True
2,gatccttgactaccgactggtAAACTTATACTTAATAATTAGAAGTTACATATCATCAGCTGTGTAatgatcctgacgacggagaccgccgtcgtcgacaagccAAGCATGGTAACCTCTCATCTTACTTATGAAATTTTAATGTATTCgatcggaatacgaccctctt,orbit_TF_del_avd_ovlp_short,True
3,gatccttgactaccgactggtTCGAAGAGAGACACTACCTGCAACAATCAGGAGCGCAATATGTCAggcttgtcgacgacggcggtctccgtcgtcaggatcatAGTAAGAACATTTGCAGTTAAAAATTTAGCTAAACACATATGAATgatcggaatacgaccctctt,orbit_TF_del_avd_ovlp_short,True
4,gatccttgactaccgactggtTAAAATAGAATGAATCATCAATCCGCATAAGAAAATCCTATGGAAggcttgtcgacgacggcggtctccgtcgtcaggatcatATGCGTACCATCAAGCCCTGATCGGCTTTTTTAATCCCATACTTTgatcggaatacgaccctctt,orbit_TF_del_avd_ovlp_short,True
...,...,...,...
69,gatccttgactaccgactggtATATTCAAGAATGAAAAATTTTTGTCATTCCTTATGCTCCTTACAatgatcctgacgacggagaccgccgtcgtcgacaagccCGCCATTACCAATACCTTACATATATTACTCATTAATGTATGTGCgatcggaatacgaccctctt,orbit_TF_del_avd_ovlp_short,True
70,gatccttgactaccgactggtTGAGTGTCGAATCCTTATCCAAAACAAGAGGTAACTCTCATGCTTggcttgtcgacgacggcggtctccgtcgtcaggatcatAATCTCAAAAGACGATACTGAACAAATTTTATCAGGTGACGTTCCgatcggaatacgaccctctt,orbit_TF_del_avd_ovlp_short,True
71,gatccttgactaccgactggtAGATAAGCTCAGCGCATCCGGCAGTTATGCCGCACGTTCATCCCGatgatcctgacgacggagaccgccgtcgtcgacaagccACTCATTTCATACTTACCTTTTTGTACGTACTTACTAAAAGTAAGgatcggaatacgaccctctt,orbit_TF_del_avd_ovlp_short,True
72,gatccttgactaccgactggtGTGTAATCCTGCCAGTGCAAAAAATCAACAACCACTCTTAACGCCatgatcctgacgacggagaccgccgtcgtcgacaagccATACATACATACTCCACTAGTTATCGTTGATTTTGTCCAACAACTgatcggaatacgaccctctt,orbit_TF_del_avd_ovlp_short,True


In [67]:
ends_4 = df_fwd_rev.iloc[3,:].str.lower()

df_4['sequence'] = ends_4['fwd_seq_t'] + df_4['oligo'] + ends_4['rev_seq_comp']

df_4['pool_name'] = 'orbit_TF_del_avd_ovlp_long'
df_4['primers_added'] = True

df_4_clean = df_4[['sequence','pool_name','primers_added']]
df_4_clean

,sequence,pool_name,primers_added
0,atttctccactcccaactggtTATTATGTGATCTAAATCACTTTTAAGTCAGAGTGAATAATGGAAggcttgtcgacgacggcggtctccgtcgtcaggatcatGGGCGCGGGAAAGAGAAGTAATTCATATTGTACTGTTACGTTGTAgatccatcgtccaaaagctg,orbit_TF_del_avd_ovlp_long,True
1,atttctccactcccaactggtCAGACTCTATTTTTTTATGCAGTTTTAACTTTGCAGATAGCCGCAatgatcctgacgacggagaccgccgtcgtcgacaagccAGCCATGACGCCACCGATAACCGTTATTTATCAGACCAAAGAAACgatccatcgtccaaaagctg,orbit_TF_del_avd_ovlp_long,True
2,atttctccactcccaactggtAAAATCCTGGAGTCAGATTCAGGGTTATTCGTTAGTGGCAGGATTatgatcctgacgacggagaccgccgtcgtcgacaagccTGCCATATGTTCGTGAATTTACAGGCGTTAGATTTACATACATTTgatccatcgtccaaaagctg,orbit_TF_del_avd_ovlp_long,True
3,atttctccactcccaactggtCAGCATCGGCAAACAGATCCAACATTACCTCTCCTCATTTTCAGCatgatcctgacgacggagaccgccgtcgtcgacaagccTTTCATAATCAGCTCCCTGGTTAAGGATAGCCTTTAGGCTGCCCGgatccatcgtccaaaagctg,orbit_TF_del_avd_ovlp_long,True
4,atttctccactcccaactggtGCGAGAACTGGTCTTTTATTCGCACTCAGGAGTACATGTATGAGGggcttgtcgacgacggcggtctccgtcgtcaggatcatAGAGAACGCACTGTCGCCTGATTTTTAACCTTAACGAAGAGCTATgatccatcgtccaaaagctg,orbit_TF_del_avd_ovlp_long,True
...,...,...,...
221,atttctccactcccaactggtGAATTTCGCCAGTTAATGCATCTTTAATCGGGAACTTTCATGAAAggcttgtcgacgacggcggtctccgtcgtcaggatcatAGCGCCCGTTTTCAGGGCTAACGTCAGAAGGTTAATTCTGTTTCCgatccatcgtccaaaagctg,orbit_TF_del_avd_ovlp_long,True
222,atttctccactcccaactggtGGAATAATCACGCAATTAACTAAACAAGGTTTAGTGAAGATGAGAggcttgtcgacgacggcggtctccgtcgtcaggatcatGCGCAGTTACGACAGATTTGATGGCGCGATAACGTAGAAAGGCTTgatccatcgtccaaaagctg,orbit_TF_del_avd_ovlp_long,True
223,atttctccactcccaactggtTGTGATCTCCATTTCGATTGATTTAGTGTTTATTGACGTATGTACggcttgtcgacgacggcggtctccgtcgtcaggatcatCGTGAGGTTAATCGTGATTGATTATAAAAAAAACTTATTATTTATgatccatcgtccaaaagctg,orbit_TF_del_avd_ovlp_long,True
224,atttctccactcccaactggtCCGGTTGCTGTACCGGGAACGTATTTAATTCCCCTGCATCGCCCGatgatcctgacgacggagaccgccgtcgtcgacaagccTAGCATTTCTCCAGCACTCTGGAGAAATAGGCAAGACATTGGCAGgatccatcgtccaaaagctg,orbit_TF_del_avd_ovlp_long,True


In [68]:
df_1_clean.to_csv("../../../../data/twist_order/twist_orbit_TF_del_first_last_short.csv")
df_2_clean.to_csv("../../../../data/twist_order/twist_orbit_TF_del_first_last_long.csv")
df_3_clean.to_csv("../../../../data/twist_order/twist_orbit_TF_del_avd_ovlp_short.csv")
df_4_clean.to_csv("../../../../data/twist_order/twist_orbit_TF_del_avd_ovlp_long.csv")

# Computational Environment

In [69]:
%load_ext watermark
%watermark -v -p wgregseq,numpy,pandas

Python implementation: CPython
Python version       : 3.8.5
IPython version      : 7.10.0

wgregseq: 0.0.1
numpy   : 1.18.1
pandas  : 1.1.5

